# RNN in TensorFlow

<img src="Images/lab12_1.png">

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:
# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

<img src="Images/lab12_2.png">

In [3]:
with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

2 2
array([[[ 1.,  0.,  0.,  0.]]], dtype=float32)
array([[[-0.60999197, -0.48648444]]], dtype=float32)


<img src="Images/lab12_3.png">

In [4]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 2
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32) #만들어 놓은 데이터를 그대로 가져와서 배열로
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

(1, 5, 4)
array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]], dtype=float32)
array([[[-0.42195684,  0.45124537],
        [ 0.41948298,  0.68618923],
        [ 0.01648408, -0.06013088],
        [ 0.54769254,  0.15267676],
        [ 0.09672112, -0.61177731]]], dtype=float32)


<img src="Images/lab12_4.png">
<img src="Images/lab12_5.png">

In [5]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data) #배치 사이즈 3
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[ -5.68959191e-02,  -2.67997719e-02],
        [ -4.95099872e-02,   2.26586638e-03],
        [ -6.98710233e-02,  -1.31108155e-02],
        [ -7.63610303e-02,  -1.93844885e-02],
        [  1.03272557e-01,  -1.11968219e-01]],

       [[ -8.17741529e-05,   1.52112786e-02],
        [  1.48568094e-01,  -9.31976810e-02],
        [  1.00002185e-01,  -8.42545107e-02],
        [  4.93090078e-02,  -7.55852535e-02],
        [  1.26621583e-02,  -6.54509142e-02]],

       [[ -3.43160927e-02,  -1.85822099e-02],
        [ -5.292172

<img src="Images/lab12_6.png">
<img src="Images/lab12_7.png">

In [6]:
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

num_classes = 5
input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
learning_rate = 0.1

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])  # X one-hot
#6개의 sequence가 필요하고(각각의 입력에 대해 i, h, e, l, l, o를 출력할 LSTM 셀), 5개의 단어가 필요하다. (h, i, e, l, o) -  one-hot
#(NONE, 6, 5)의 입력 shape
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
#정답 레이블
#tf.placeholder()는 입력 데이터를 만들 때 주로 사용한다. (실제 훈련 예제를 제공하는 변수) - 초기값을 지정할 필요 없다. (모델 입력시 변경되지 않을 데이터)
#tf.Variable()은 데이터의 상태를 저장할 때 주로 사용한다. (가중치나 편향 등의 학습 가능한 변수) - 초기값을 지정해야 한다. (학습 되는 데이터)
#http://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
#num_units는 출력값 = 5 (h, i, e, l, o). 튜플로 state 반환
initial_state = cell.zero_state(batch_size, tf.float32)
#초기화
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
#dynamic_rnn으로 output을 받는다. (cell, 입력, 초기값, 타입)

#보통은 dynamic_rnn의 결과값을 바로 예측에 사용하지 않는다. 다시 FC하는 것이 일반적
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
#결과를  reshape해서 평평하게. FC 전에 평평하게 펴줘야.
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc, num_outputs=num_classes, activation_fn=None)
#FC로 연결(tf.layers 사용) (인풋, 아웃풋 수)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
#아웃풋을 다시 reshape RNN결과에 맞게.

weights = tf.ones([batch_size, sequence_length])
#일반적인 딥러닝의 가중치가 아니라, 각각의 자리를 얼마나 중요하게 생각하느냐의 값. 여기서는 모두 같으므로 1로 가득찬 가중치를 주면 된다.
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
#sequence_loss를 통해 cost를 계산한다. (로짓 - 예상치, 정답레이블, 가중치)
loss = tf.reduce_mean(sequence_loss) #평균
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) #Adam으로 최적화

prediction = tf.argmax(outputs, axis=2) #예상값

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #초기화
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data}) #cost값을 구해야 하므로 정답레이블이 필요하다.
        result = sess.run(prediction, feed_dict={X: x_one_hot}) #예상값만 추려내므로 정답레이블이 필요없다.
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)] #result를 다시 문자열로 바꾼다.
        print("\tPrediction str: ", ''.join(result_str))

0 loss: 1.56977 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
1 loss: 1.44971 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
2 loss: 1.33331 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  llllll
3 loss: 1.19909 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
4 loss: 1.06452 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
5 loss: 0.960249 prediction:  [[2 2 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  eeelll
6 loss: 0.864713 prediction:  [[0 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  hhello
7 loss: 0.790663 prediction:  [[0 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  hhello
8 loss: 0.730812 prediction:  [[0 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  hhello
9 loss: 0.662183 prediction:  [[0 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Prediction str:  hh

# RNN with sentence

In [6]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
#인덱스를 입력하면 문자를 반환해주는 List. Set은 중복을 허용하지 않기 때문에 set에 담은 후 다시 list로 변환해서 만든다.
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
#문자를 입력하면 인덱스를 반환해주는 Dictionary. key(c): value(i).
#enum으로 인덱스와 값을 가져와 각각 Dictionary에 저장.

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
#문자의 총 길이 (중복 배제한 문자의 수)
hidden_size = len(char2idx)  # RNN output size
#RNN의 output. 문자의 수와 같다.
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
#최종 output. 문자의 수와 같다.
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
#LSTM 셀의 총 수. 처음 input에서 2번째의 값이 추론되므로 하나씩 밀려서 정답 문자열의 총 길이 -1이 된다.
learning_rate = 0.1 #학습률

sample_idx = [char2idx[c] for c in sample]  # char to index
#sample 문자열을 인덱스로 변환. 문자를 인덱스로 반환해주는 Dictionary에 넣어 각 sample문자의 index를 담은 리스트를 만든다.
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
#input 데이터 0 ~ n-1까지로 Slicing
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello
#정답 레이블. 1 ~ n까지로 Slicing

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label
#tf.placeholder()는 입력 데이터를 만들 때 주로 사용한다. (실제 훈련 예제를 제공하는 변수) - 초기값을 지정할 필요 없다. (모델 입력시 변경되지 않을 데이터)
#tf.Variable()은 데이터의 상태를 저장할 때 주로 사용한다. (가중치나 편향 등의 학습 가능한 변수) - 초기값을 지정해야 한다. (학습 되는 데이터)
#http://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
#tf.one_hot으로 one-hot encoding. (입력데이터, 총 수)
#tf.one_hot은 항상 shape에 주의해야한다. one_hot이후 shape 확인해 볼 것.
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
#LSTM cell 생성. 아웃풋은 hidden_size, 튜플로 state 반환
initial_state = cell.zero_state(batch_size, tf.float32) #상태 초기화
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)
#dynamic_rnn로 RNN의 output을 가져온다. (LSTM cell, 인풋데이터, 상태 초기화)

# FC layer
#보통은 dynamic_rnn의 결과값을 바로 예측에 사용하지 않는다. 다시 FC하는 것이 일반적
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
#결과를  reshape해서 평평하게. FC 전에 평평하게 펴줘야 한다.
outputs = tf.contrib.layers.fully_connected(outputs, num_classes, activation_fn=None)
#FC로 연결(tf.layers 사용) (인풋, 아웃풋 수)
#tf.layers 대신 아래와 같이 써도 된다.
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b

#activation_fn: 활성화 기능. 기본값은 ReLU. 명시적으로 None으로 설정하면 건너 뛰고 선형 활성화를 유지한다.

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
#아웃풋을 다시 reshape RNN결과에 맞게.

weights = tf.ones([batch_size, sequence_length])
#일반적인 딥러닝의 가중치가 아니라, 각각의 자리를 얼마나 중요하게 생각하느냐의 값. 여기서는 모두 같으므로 1로 가득찬 가중치를 주면 된다.
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
#sequence_loss를 통해 cost를 계산한다. (로짓 - 예상치, 정답레이블, 가중치)
loss = tf.reduce_mean(sequence_loss) #평균
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
#Adam으로 최적화

prediction = tf.argmax(outputs, axis=2) #인덱스로 예상값를 구한다.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #초기화
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data}) #cost값을 구해야 하므로 정답레이블이 필요하다.
        result = sess.run(prediction, feed_dict={X: x_data}) #예상값만 추려내므로 정답레이블이 필요없다.

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)] #result를 다시 문자열로 바꾼다.

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.31387 Prediction: y   uuuu      u
1 loss: 2.17669 Prediction: o   oooo    ooo
2 loss: 2.03696 Prediction:     ou o     ou
3 loss: 1.89293 Prediction:     ouuu     ou
4 loss: 1.67043 Prediction:     ouuu     ou
5 loss: 1.40496 Prediction: y  youuwant you
6 loss: 1.14893 Prediction: y  youuwant you
7 loss: 0.920597 Prediction: yf youuwant you
8 loss: 0.704134 Prediction: yf you want you
9 loss: 0.536923 Prediction: if you want you
10 loss: 0.397025 Prediction: if you want you
11 loss: 0.29825 Prediction: if you want you
12 loss: 0.215481 Prediction: if you want you
13 loss: 0.161236 Prediction: if you want you
14 loss: 0.117642 Prediction: if you want you
15 loss: 0.0848477 Prediction: if you want you
16 loss: 0.0631928 Prediction: if you want you
17 loss: 0.0478475 Prediction: if you want you
18 loss: 0.0364256 Prediction: if you want you
19 loss: 0.0277816 Prediction: if you want you
20 loss: 0.0212375 Prediction: if you want you
21 loss: 0.0162933 Prediction: if you want you

# RNN with long sequences

<img src="Images/lab12_8.png">

In [6]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.") #학습할 문자열

char_set = list(set(sentence)) # index -> char
#인덱스를 입력하면 문자를 반환해주는 List. Set은 중복을 허용하지 않기 때문에 set에 담은 후 다시 list로 변환해서 만든다.
char_dic = {w: i for i, w in enumerate(char_set)} # char -> idex
#문자를 입력하면 인덱스를 반환해주는 Dictionary. key(c): value(i).
#enum으로 인덱스와 값을 가져와 각각 Dictionary에 저장.

data_dim = len(char_set) # RNN input size (one hot size)
#문자의 총 길이 (중복 배제한 문자의 수)
hidden_size = len(char_set) # RNN output size
#RNN의 output. 문자의 수와 같다.
num_classes = len(char_set) # final output size (RNN or softmax, etc.) 
#최종 output. 문자의 수와 같다.
sequence_length = 10  # Any arbitrary number #이 변수만큼 문자열을 잘라서 단계적으로 학습한다. # number of lstm rollings (unit #)
#LSTM 셀의 총 수. 
learning_rate = 0.1 #학습률

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length): #sequence_length만큼씩 잘라서 data set을 만든다. ?? 그냥 해도 되지 않나?
    x_str = sentence[i:i + sequence_length] #i ~ i + sequence_length -1 만큼 자른다.
    y_str = sentence[i + 1: i + sequence_length + 1] #i + 1 ~ i + sequence_length + 1만큼 자른다.
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    #문자열을 인덱스로 변환. 문자를 인덱스로 반환해주는 Dictionary에 넣어 각 sentence문자의 index를 담은 리스트를 만든다. : 인덱스
    y = [char_dic[c] for c in y_str]  # y str to index
    #문자열을 인덱스로 변환. 문자를 인덱스로 반환해주는 Dictionary에 넣어 각 sentence문자의 index를 담은 리스트를 만든다. : 정답레이블

    dataX.append(x) #인덱스에 추가
    dataY.append(y) #정답 레이블에 추가

batch_size = len(dataX) #중간에 잘라도 되지만, 한 번에 모든 문자를 학습

X = tf.placeholder(tf.int32, [None, sequence_length]) #sequence_length만큼씩. 배치 사이즈
Y = tf.placeholder(tf.int32, [None, sequence_length]) #sequence_length만큼씩. 배치 사이즈
#tf.placeholder()는 입력 데이터를 만들 때 주로 사용한다. (실제 훈련 예제를 제공하는 변수) - 초기값을 지정할 필요 없다. (모델 입력시 변경되지 않을 데이터)
#tf.Variable()은 데이터의 상태를 저장할 때 주로 사용한다. (가중치나 편향 등의 학습 가능한 변수) - 초기값을 지정해야 한다. (학습 되는 데이터)
#http://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes) #tf.one_hot으로 one-hot encoding. (입력데이터, 총 수)
#tf.one_hot은 항상 shape에 주의해야한다. one_hot이후 shape 확인해 볼 것.
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell(): #BasicLSTMCell을 만드는 함수
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)
#셀을 전달하면 자동으로 입력 또는 출력에 누락이 추가됩니다. 마지막으로 tf.contrib.rnn.MultiRNNCell을 사용하여 LSTM 셀을 레이어에 쌓을 수 있습니다. 이렇게하면 셀 목록을 전달하고 한 셀의 출력을 다음 셀로 보냅니다.
#lstm_cell * 2 만큼 셀 생성. 동일한 셀 객체를 생성(리턴) 하지만 모든 셀 객체에 대해 서로 다른 가중치 행렬을 만든다.
#lstm_cell * 2 오류??

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)
#dynamic_rnn로 RNN의 output을 가져온다. (LSTM cell, 인풋데이터, 상태 초기화)

# FC layer
#보통은 dynamic_rnn의 결과값을 바로 예측에 사용하지 않는다. 다시 FC하는 것이 일반적
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
#결과를  reshape해서 평평하게. FC 전에 평평하게 펴줘야 한다.
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)
#FC로 연결(tf.layers 사용) (인풋, 아웃풋 수)
#tf.layers 대신 아래와 같이 써도 된다.
# fc_w = tf.get_variable("fc_w", [hidden_size, num_classes])
# fc_b = tf.get_variable("fc_b", [num_classes])
# outputs = tf.matmul(X_for_fc, fc_w) + fc_b

#activation_fn: 활성화 기능. 기본값은 ReLU. 명시적으로 None으로 설정하면 건너 뛰고 선형 활성화를 유지한다.

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
#아웃풋을 다시 reshape RNN결과에 맞게.

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])
#일반적인 딥러닝의 가중치가 아니라, 각각의 자리를 얼마나 중요하게 생각하느냐의 값. 여기서는 모두 같으므로 1로 가득찬 가중치를 주면 된다.

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
#sequence_loss를 통해 cost를 계산한다. (로짓 - 예상치, 정답레이블, 가중치)
#여기서 output은 softmax를 거치기 전의 값. RNN LSTM에는 활선함수가 포함되어 있기 때문에 그 결과를 바로 loss 함수에 적용하면 잘 학습이 되지 않는다.
#contrib.seq2seq.sequence_loss는 cross entropy의 손실 값을 구하는 함수

mean_loss = tf.reduce_mean(sequence_loss) #평균
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)
#Adam으로 최적화

sess = tf.Session() #세션 생성
sess.run(tf.global_variables_initializer()) #초기화

for i in range(500):
    _, l, results = sess.run([train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

1 59            3.01384
1 60            3.01384
1 61            3.01384
1 62     o      3.01384
1 63    ooooo   3.01384
1 64   ooooo    3.01384
1 65     ooo    3.01384
1 66    ooo     3.01384
1 67    o       3.01384
1 68   oo       3.01384
1 69  nn        3.01384
1 70 nn         3.01384
1 71            3.01384
1 72            3.01384
1 73            3.01384
1 74            3.01384
1 75            3.01384
1 76            3.01384
1 77   nn       3.01384
1 78  n         3.01384
1 79            3.01384
1 80            3.01384
1 81            3.01384
1 82            3.01384
1 83            3.01384
1 84    nnnnn   3.01384
1 85   nnnnn    3.01384
1 86 nnnnnn     3.01384
1 87 nn         3.01384
1 88            3.01384
1 89            3.01384
1 90            3.01384
1 91            3.01384
1 92     oo     3.01384
1 93   ooooo    3.01384
1 94   ooo      3.01384
1 95            3.01384
1 96        nn  3.01384
1 97       nnnn 3.01384
1 98    nnnnnnn 3.01384
1 99 nnnnnnnnnn 3.01384
1 100   nnnnnnn 

4 104            2.91296
4 105            2.91296
4 106            2.91296
4 107            2.91296
4 108            2.91296
4 109            2.91296
4 110            2.91296
4 111            2.91296
4 112            2.91296
4 113            2.91296
4 114            2.91296
4 115            2.91296
4 116            2.91296
4 117            2.91296
4 118            2.91296
4 119            2.91296
4 120            2.91296
4 121            2.91296
4 122            2.91296
4 123            2.91296
4 124            2.91296
4 125            2.91296
4 126            2.91296
4 127            2.91296
4 128            2.91296
4 129            2.91296
4 130            2.91296
4 131            2.91296
4 132            2.91296
4 133            2.91296
4 134            2.91296
4 135            2.91296
4 136            2.91296
4 137            2.91296
4 138            2.91296
4 139            2.91296
4 140            2.91296
4 141            2.91296
4 142            2.91296
4 143            2.91296


8 0 o          2.84911
8 1 ooooo      2.84911
8 2 oooo       2.84911
8 3  oo        2.84911
8 4            2.84911
8 5            2.84911
8 6 oo         2.84911
8 7 o          2.84911
8 8            2.84911
8 9   oo       2.84911
8 10 ooooo      2.84911
8 11 oooo       2.84911
8 12 oooo       2.84911
8 13  o         2.84911
8 14 o          2.84911
8 15            2.84911
8 16            2.84911
8 17 o          2.84911
8 18 o          2.84911
8 19            2.84911
8 20 o          2.84911
8 21            2.84911
8 22 o          2.84911
8 23            2.84911
8 24 ooo        2.84911
8 25 oo         2.84911
8 26 oooo       2.84911
8 27            2.84911
8 28 o          2.84911
8 29            2.84911
8 30            2.84911
8 31            2.84911
8 32            2.84911
8 33 ooo        2.84911
8 34 o          2.84911
8 35            2.84911
8 36            2.84911
8 37          o 2.84911
8 38         oo 2.84911
8 39 o oooooooo 2.84911
8 40   oooooooo 2.84911
8 41 oooooooooo 2.84911
8 

10 127  ooooooooo 2.71781
10 128  ooooooooo 2.71781
10 129  ooo  oooo 2.71781
10 130  oo  o ooo 2.71781
10 131  o     o   2.71781
10 132            2.71781
10 133            2.71781
10 134  oooooo oo 2.71781
10 135  oooo  ooo 2.71781
10 136   o   t oo 2.71781
10 137  oo     oo 2.71781
10 138         oo 2.71781
10 139    t   too 2.71781
10 140        ooo 2.71781
10 141    ooooooo 2.71781
10 142   tooooooo 2.71781
10 143    ooooooo 2.71781
10 144   toooooo  2.71781
10 145   ooooo    2.71781
10 146  ooooo     2.71781
10 147  oooo      2.71781
10 148  oo        2.71781
10 149            2.71781
10 150            2.71781
10 151            2.71781
10 152            2.71781
10 153            2.71781
10 154            2.71781
10 155            2.71781
10 156            2.71781
10 157            2.71781
10 158  o         2.71781
10 159            2.71781
10 160            2.71781
10 161            2.71781
10 162            2.71781
10 163            2.71781
10 164            2.71781
10 165      

...

487 107 ook, but r 0.228996
487 108 nk, but ra 0.228996
487 109  , but rat 0.228996
487 110 s but rath 0.228996
487 111  but rathe 0.228996
487 112 tut rather 0.228996
487 113 ut rather  0.228996
487 114 t rather t 0.228996
487 115 hdather te 0.228996
487 116 tather tea 0.228996
487 117  ther teac 0.228996
487 118 nher teach 0.228996
487 119 her teach  0.228996
487 120 er toach t 0.228996
487 121 n teach th 0.228996
487 122  thach the 0.228996
487 123 toach them 0.228996
487 124 hach them  0.228996
487 125 nch them t 0.228996
487 126 nh them to 0.228996
487 127 o them to  0.228996
487 128 ethem to l 0.228996
487 129 toem ta lo 0.228996
487 130 her ta lon 0.228996
487 131 er ta long 0.228996
487 132 n ta long  0.228996
487 133  ta long f 0.228996
487 134 to cong fo 0.228996
487 135 h cong for 0.228996
487 136 ncong for  0.228996
487 137 tong for t 0.228996
487 138 eng for th 0.228996
487 139 n' for the 0.228996
487 140 d for the  0.228996
487 141  for the e 0.228996
487 142 tor the en 0

490 79  n't assig 0.228987
490 80 n't dssign 0.228987
490 81 dt assign  0.228987
490 82 t assign t 0.228987
490 83 htssign th 0.228987
490 84 tnsign the 0.228987
490 85 nkign them 0.228987
490 86   gn them  0.228987
490 87  gn them t 0.228987
490 88 gn them ta 0.228987
490 89   them tas 0.228987
490 90 dthem task 0.228987
490 91 toem tasks 0.228987
490 92 her tasks  0.228987
490 93 er tasks a 0.228987
490 94   tasks an 0.228987
490 95  tasks and 0.228987
490 96 tosks and  0.228987
490 97 hsks and w 0.228987
490 98 nks and wo 0.228987
490 99  s and wor 0.228987
490 100 , and work 0.228987
490 101  and work, 0.228987
490 102 tnd work,  0.228987
490 103 nd work, b 0.228987
490 104 d dork, bu 0.228987
490 105  aork, but 0.228987
490 106 took, but  0.228987
490 107 ook, but r 0.228987
490 108 nk, but ra 0.228987
490 109  , but rat 0.228987
490 110 , but rath 0.228987
490 111  dut rathe 0.228987
490 112 tut rather 0.228987
490 113 ut rather  0.228987
490 114 t rather t 0.228987
490 115 htath

492 139  ' for the 0.228946
492 140 d for the  0.228946
492 141 nfor the e 0.228946
492 142 tor the en 0.228946
492 143 or the end 0.228946
492 144   the endl 0.228946
492 145  toe endle 0.228946
492 146 toemendles 0.228946
492 147 hemendless 0.228946
492 148 emendless  0.228946
492 149 mtndless i 0.228946
492 150 tndless im 0.228946
492 151 msless imm 0.228946
492 152 d ess imme 0.228946
492 153  ess immen 0.228946
492 154 ess immens 0.228946
492 155 ms immensi 0.228946
492 156 siimmensit 0.228946
492 157 simmensity 0.228946
492 158 tmmensity  0.228946
492 159 tmensity o 0.228946
492 160  ensity of 0.228946
492 161  nsity of  0.228946
492 162 msity of t 0.228946
492 163 dity of th 0.228946
492 164 sty of the 0.228946
492 165 ty of the  0.228946
492 166 h of the s 0.228946
492 167 oof the se 0.228946
492 168 tf the sea 0.228946
492 169   the sea. 0.228946
493 0 t you want 0.228959
493 1  you want  0.228959
493 2 tou want t 0.228959
493 3 ou want to 0.228959
493 4 n want to  0.228959
49

495 53 mher to co 0.228937
495 54 hem te col 0.228937
495 55 em te coll 0.228937
495 56 m te colle 0.228937
495 57  to collec 0.228937
495 58 to lollect 0.228937
495 59 h lollect  0.228937
495 60 ncollect w 0.228937
495 61 tollect wo 0.228937
495 62 tllect woo 0.228937
495 63 nlect wood 0.228937
495 64 eect wood  0.228937
495 65 ect wood a 0.228937
495 66 mt wood an 0.228937
495 67 t wood and 0.228937
495 68 hwood and  0.228937
495 69 tood and d 0.228937
495 70 ood and do 0.228937
495 71 nd and don 0.228937
495 72 n and don' 0.228937
495 73  a d don't 0.228937
495 74 tnd won't  0.228937
495 75 nd won't a 0.228937
495 76 d won't as 0.228937
495 77  aon't ass 0.228937
495 78 ton't assi 0.228937
495 79  n't assig 0.228937
495 80 n't assign 0.228937
495 81 dt assign  0.228937
495 82 t assign t 0.228937
495 83 hwssign th 0.228937
495 84 tnsign the 0.228937
495 85 nkign them 0.228937
495 86 i gn them  0.228937
495 87 itn them t 0.228937
495 88 pn them ta 0.228937
495 89 e them tas 0.228937
4

497 117  ther teac 0.228929
497 118 nher teach 0.228929
497 119  er toach  0.228929
497 120 er toach t 0.228929
497 121 r toach th 0.228929
497 122  thach the 0.228929
497 123 toach them 0.228929
497 124  ach them  0.228929
497 125 rch them t 0.228929
497 126 nh them to 0.228929
497 127 o them to  0.228929
497 128 ethem to l 0.228929
497 129 toem to lo 0.228929
497 130  er to lon 0.228929
497 131 er to long 0.228929
497 132 r to long  0.228929
497 133  to long f 0.228929
497 134 to long fo 0.228929
497 135   bong for 0.228929
497 136  bong for  0.228929
497 137 tong for t 0.228929
497 138 eng for th 0.228929
497 139  ' for the 0.228929
497 140 d for the  0.228929
497 141 nfor the e 0.228929
497 142 tor the en 0.228929
497 143  r the end 0.228929
497 144   the endl 0.228929
497 145  the endle 0.228929
497 146 toemendles 0.228929
497 147  erendless 0.228929
497 148 erendless  0.228929
497 149 rendless i 0.228929
497 150 tndless im 0.228929
497 151 rsless imm 0.228929
497 152 d ess imme 0

# Dynamic RNN

이전의 모델은 RNN의 input값을 고정시켜야 한다는 단점이 있다. Dynamic RNN을 이용하면 input의 수를 유동적으로 바꿀 수 있다.

In [4]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(
        cell, x_data, sequence_length=[5,3,4], dtype=tf.float32) #적용. 
    #3의 경우 나머지 2개의 요소를 0으로 출력하고, 4의 경우에는 나머지 1개의 요소를 0으로 출력한다.
    #일반적으로 padding을 넣는 법이 있었지만, loss함수를 구할 때 값이 뒤섞여 제대로 학습이 되지 않을 수 있다.
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]],

       [[ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.]]], dtype=float32)
array([[[-0.04331283,  0.07370758],
        [-0.10487618,  0.11025281],
        [-0.12882552,  0.17869201],
        [-0.15873177,  0.24327616],
        [-0.17179659,  0.18555647]],

       [[-0.04522783,  0.02267111],
        [-0.08628279,  0.06642862],
        [-0.12212723,  0.17323987],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[-0.06000253,  0.10318186],
        [-0.10730334,  0.18585774],
        [-0.15988328,  0.21667241],
        [-0.18693614,  0.21268262],
        [ 0.        ,

# RNN with Time Series Data

In [5]:
'''
This script shows how to predict stock prices using a basic RNN
'''
import tensorflow as tf
import numpy as np
import matplotlib
import os

tf.set_random_seed(777)  # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt


def MinMaxScaler(data):
    ''' Min Max Normalization
    Parameters
    ----------
    data : numpy.ndarray
        input data to be normalized
        shape: [Batch size, dimension]
    Returns
    ----------
    data : numpy.ndarry
        normalized data
        shape: [Batch size, dimension]
    References
    ----------
    .. [1] http://sebastianraschka.com/Articles/2014_about_feature_scaling.html
    '''
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)


# train Parameters
seq_length = 7 #7일치 만큼 끊어서
data_dim = 5 #
hidden_dim = 10
output_dim = 1 #1개만 예측하므로 
learning_rate = 0.01
iterations = 500

# Open, High, Low, Volume, Close
xy = np.loadtxt('data-02-stock_daily.csv', delimiter=',')
xy = xy[::-1]  # reverse order (chronically ordered)
xy = MinMaxScaler(xy)
x = xy
y = xy[:, [-1]]  # Close as label

# build a dataset
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

# train/test split
train_size = int(len(dataY) * 0.7) #70% 데이터 학습, 30% 예측
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(
    dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(
    dataY[train_size:len(dataY)])

# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

# RMSE
targets = tf.placeholder(tf.float32, [None, 1])
predictions = tf.placeholder(tf.float32, [None, 1])
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))

    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    rmse_val = sess.run(rmse, feed_dict={
                    targets: testY, predictions: test_predict})
    print("RMSE: {}".format(rmse_val))

    # Plot predictions
    plt.plot(testY)
    plt.plot(test_predict)
    plt.xlabel("Time Period")
    plt.ylabel("Stock Price")
    plt.show()

[[  2.13751054e-01   2.08179810e-01   1.91791832e-01   4.66075110e-04
    1.92092403e-01]
 [  1.93935034e-01   2.03641926e-01   2.08664571e-01   2.98467330e-03
    1.96551555e-01]
 [  2.10516454e-01   2.05289413e-01   2.03558748e-01   2.59926504e-04
    1.87749731e-01]
 [  1.86678765e-01   2.09480567e-01   2.08781843e-01   0.00000000e+00
    2.17448151e-01]
 [  3.06973882e-01   3.14634137e-01   2.18993665e-01   1.24764722e-02
    2.16981885e-01]
 [  2.19142110e-01   2.63907214e-01   2.24686396e-01   4.56323384e-01
    2.24967473e-01]
 [  2.33129931e-01   2.36419163e-01   1.62682724e-01   5.70171193e-01
    1.47442742e-01]] -> [ 0.13289962]
[[  1.93935034e-01   2.03641926e-01   2.08664571e-01   2.98467330e-03
    1.96551555e-01]
 [  2.10516454e-01   2.05289413e-01   2.03558748e-01   2.59926504e-04
    1.87749731e-01]
 [  1.86678765e-01   2.09480567e-01   2.08781843e-01   0.00000000e+00
    2.17448151e-01]
 [  3.06973882e-01   3.14634137e-01   2.18993665e-01   1.24764722e-02
    2.169818

 [ 0.28899382  0.28318331  0.28140806  0.15072152  0.27634948]] -> [ 0.25816326]
[[ 0.22814676  0.22826824  0.23665859  0.14247558  0.23476016]
 [ 0.22762223  0.24720044  0.24971659  0.13197096  0.25472422]
 [ 0.24874968  0.26460085  0.26766042  0.1510173   0.26877183]
 [ 0.26911948  0.27043949  0.28592698  0.12876221  0.28188707]
 [ 0.28223302  0.29049908  0.29913189  0.33420274  0.30173448]
 [ 0.28899382  0.28318331  0.28140806  0.15072152  0.27634948]
 [ 0.26868234  0.2626643   0.27418952  0.13117325  0.25816326]] -> [ 0.2781564]
[[ 0.22762223  0.24720044  0.24971659  0.13197096  0.25472422]
 [ 0.24874968  0.26460085  0.26766042  0.1510173   0.26877183]
 [ 0.26911948  0.27043949  0.28592698  0.12876221  0.28188707]
 [ 0.28223302  0.29049908  0.29913189  0.33420274  0.30173448]
 [ 0.28899382  0.28318331  0.28140806  0.15072152  0.27634948]
 [ 0.26868234  0.2626643   0.27418952  0.13117325  0.25816326]
 [ 0.2529752   0.27069971  0.27278097  0.15418123  0.2781564 ]] -> [ 0.24047238]
[[

 [ 0.06180855  0.12687224  0.08204559  0.50178363  0.12232013]] -> [ 0.10471665]
[[ 0.11930419  0.1335202   0.13333871  0.2038003   0.13814557]
 [ 0.12787188  0.12435758  0.12356725  0.13765349  0.12433111]
 [ 0.10292681  0.10036705  0.08988032  0.16994712  0.07600928]
 [ 0.08197432  0.07808186  0.06584776  0.15021063  0.05085741]
 [ 0.04764595  0.04368588  0.04002511  0.37450928  0.05278089]
 [ 0.06180855  0.12687224  0.08204559  0.50178363  0.12232013]
 [ 0.10805568  0.10701502  0.09093687  0.25471901  0.10471665]] -> [ 0.10693169]
[[ 0.12787188  0.12435758  0.12356725  0.13765349  0.12433111]
 [ 0.10292681  0.10036705  0.08988032  0.16994712  0.07600928]
 [ 0.08197432  0.07808186  0.06584776  0.15021063  0.05085741]
 [ 0.04764595  0.04368588  0.04002511  0.37450928  0.05278089]
 [ 0.06180855  0.12687224  0.08204559  0.50178363  0.12232013]
 [ 0.10805568  0.10701502  0.09093687  0.25471901  0.10471665]
 [ 0.097186    0.10817125  0.10475778  0.18201129  0.10693169]] -> [ 0.0880459]
[[

 [ 0.42291858  0.41642633  0.36611587  0.38158107  0.34952358]] -> [ 0.28287009]
[[ 0.46730044  0.47367086  0.48445919  0.09538406  0.47962473]
 [ 0.47251664  0.4780788   0.48029243  0.09355562  0.49055392]
 [ 0.48737871  0.48565166  0.48680684  0.12980192  0.47673945]
 [ 0.47193378  0.49432289  0.48894887  0.19034687  0.49064144]
 [ 0.46861183  0.48273232  0.45581982  0.25521197  0.44963509]
 [ 0.42291858  0.41642633  0.36611587  0.38158107  0.34952358]
 [ 0.22831412  0.29872895  0.22737841  0.51648293  0.28287009]] -> [ 0.26086602]
[[ 0.47251664  0.4780788   0.48029243  0.09355562  0.49055392]
 [ 0.48737871  0.48565166  0.48680684  0.12980192  0.47673945]
 [ 0.47193378  0.49432289  0.48894887  0.19034687  0.49064144]
 [ 0.46861183  0.48273232  0.45581982  0.25521197  0.44963509]
 [ 0.42291858  0.41642633  0.36611587  0.38158107  0.34952358]
 [ 0.22831412  0.29872895  0.22737841  0.51648293  0.28287009]
 [ 0.35044446  0.35110315  0.27450451  0.31640226  0.26086602]] -> [ 0.39656287]
[

[[ 0.67653362  0.69231591  0.69717237  0.15330286  0.69345802]
 [ 0.67685412  0.68303764  0.6960573   0.15354486  0.68716288]
 [ 0.6752514   0.69800993  0.69529433  0.14488662  0.70977905]
 [ 0.70462569  0.71419633  0.7290103   0.16607511  0.71470448]
 [ 0.72039096  0.71110357  0.71677399  0.26645156  0.71417973]
 [ 0.70477136  0.71254878  0.72172128  0.1638971   0.72726576]
 [ 0.70756897  0.7197748   0.73329456  0.11309492  0.72336032]] -> [ 0.71552044]
[[ 0.67685412  0.68303764  0.6960573   0.15354486  0.68716288]
 [ 0.6752514   0.69800993  0.69529433  0.14488662  0.70977905]
 [ 0.70462569  0.71419633  0.7290103   0.16607511  0.71470448]
 [ 0.72039096  0.71110357  0.71677399  0.26645156  0.71417973]
 [ 0.70477136  0.71254878  0.72172128  0.1638971   0.72726576]
 [ 0.70756897  0.7197748   0.73329456  0.11309492  0.72336032]
 [ 0.72184801  0.72185581  0.72945053  0.1276508   0.71552044]] -> [ 0.70747652]
[[ 0.6752514   0.69800993  0.69529433  0.14488662  0.70977905]
 [ 0.70462569  0.71

 [ 0.82952447  0.82692229  0.84535844  0.13025903  0.82746444]] -> [ 0.82860112]
[[ 0.82526981  0.82617075  0.84336291  0.09876311  0.84229908]
 [ 0.83573154  0.83770346  0.84119741  0.11712826  0.82320945]
 [ 0.82084048  0.82365611  0.84078082  0.14138209  0.82985431]
 [ 0.81477907  0.80956536  0.82731195  0.11391055  0.81606901]
 [ 0.81996628  0.81721055  0.83725953  0.1069732   0.82734793]
 [ 0.82952447  0.82692229  0.84535844  0.13025903  0.82746444]
 [ 0.8286211   0.82232646  0.84503552  0.09525858  0.82860112]] -> [ 0.82341349]
[[ 0.83573154  0.83770346  0.84119741  0.11712826  0.82320945]
 [ 0.82084048  0.82365611  0.84078082  0.14138209  0.82985431]
 [ 0.81477907  0.80956536  0.82731195  0.11391055  0.81606901]
 [ 0.81996628  0.81721055  0.83725953  0.1069732   0.82734793]
 [ 0.82952447  0.82692229  0.84535844  0.13025903  0.82746444]
 [ 0.8286211   0.82232646  0.84503552  0.09525858  0.82860112]
 [ 0.83054434  0.8199563   0.83097992  0.08293448  0.82341349]] -> [ 0.85506439]
[

 [ 0.8632991   0.8577629   0.87948527  0.05516716  0.86663467]] -> [ 0.87141442]
[[ 0.89098316  0.88122154  0.88831759  0.21694004  0.86922858]
 [ 0.86131735  0.87198367  0.87652152  0.10916913  0.87913778]
 [ 0.88037575  0.87484532  0.89706216  0.08220848  0.88560778]
 [ 0.87769482  0.86913965  0.87895692  0.1076275   0.88018695]
 [ 0.86755359  0.85943939  0.88329992  0.08615219  0.87056929]
 [ 0.8632991   0.8577629   0.87948527  0.05516716  0.86663467]
 [ 0.86265791  0.87256178  0.88059139  0.07001882  0.87141442]] -> [ 0.85247048]
[[ 0.86131735  0.87198367  0.87652152  0.10916913  0.87913778]
 [ 0.88037575  0.87484532  0.89706216  0.08220848  0.88560778]
 [ 0.87769482  0.86913965  0.87895692  0.1076275   0.88018695]
 [ 0.86755359  0.85943939  0.88329992  0.08615219  0.87056929]
 [ 0.8632991   0.8577629   0.87948527  0.05516716  0.86663467]
 [ 0.86265791  0.87256178  0.88059139  0.07001882  0.87141442]
 [ 0.87145858  0.86206958  0.86751296  0.10081563  0.85247048]] -> [ 0.84588379]
[

[step: 17] loss: 0.9418917298316956
[step: 18] loss: 0.8798741698265076
[step: 19] loss: 0.9798906445503235
[step: 20] loss: 1.066758632659912
[step: 21] loss: 1.0137057304382324
[step: 22] loss: 0.8966498374938965
[step: 23] loss: 0.8558007478713989
[step: 24] loss: 0.9103301167488098
[step: 25] loss: 0.9615064859390259
[step: 26] loss: 0.9335518479347229
[step: 27] loss: 0.8584058880805969
[step: 28] loss: 0.820594310760498
[step: 29] loss: 0.8467499613761902
[step: 30] loss: 0.8767358660697937
[step: 31] loss: 0.8539502620697021
[step: 32] loss: 0.8035181164741516
[step: 33] loss: 0.7857146263122559
[step: 34] loss: 0.8073508739471436
[step: 35] loss: 0.820772111415863
[step: 36] loss: 0.7979164719581604
[step: 37] loss: 0.7664017677307129
[step: 38] loss: 0.7628548741340637
[step: 39] loss: 0.7775619029998779
[step: 40] loss: 0.7737617492675781
[step: 41] loss: 0.7487120628356934
[step: 42] loss: 0.733856201171875
[step: 43] loss: 0.7398050427436829
[step: 44] loss: 0.7424824833869

[step: 240] loss: 0.4249121844768524
[step: 241] loss: 0.4521394371986389
[step: 242] loss: 0.45070117712020874
[step: 243] loss: 0.4085176885128021
[step: 244] loss: 0.4137846827507019
[step: 245] loss: 0.43525972962379456
[step: 246] loss: 0.4077616035938263
[step: 247] loss: 0.4107825756072998
[step: 248] loss: 0.42348816990852356
[step: 249] loss: 0.4025043249130249
[step: 250] loss: 0.41333258152008057
[step: 251] loss: 0.4126538038253784
[step: 252] loss: 0.4017646908760071
[step: 253] loss: 0.4135817885398865
[step: 254] loss: 0.4035811424255371
[step: 255] loss: 0.40532514452934265
[step: 256] loss: 0.4084668457508087
[step: 257] loss: 0.4004782736301422
[step: 258] loss: 0.4072185456752777
[step: 259] loss: 0.4021686017513275
[step: 260] loss: 0.4022325277328491
[step: 261] loss: 0.4046476185321808
[step: 262] loss: 0.39963793754577637
[step: 263] loss: 0.40349090099334717
[step: 264] loss: 0.4007788300514221
[step: 265] loss: 0.4002569615840912
[step: 266] loss: 0.40206590294

[step: 476] loss: 0.37850090861320496
[step: 477] loss: 0.3784258961677551
[step: 478] loss: 0.3783528506755829
[step: 479] loss: 0.37827974557876587
[step: 480] loss: 0.37820544838905334
[step: 481] loss: 0.3781302571296692
[step: 482] loss: 0.37805625796318054
[step: 483] loss: 0.377983033657074
[step: 484] loss: 0.3779084384441376
[step: 485] loss: 0.37783369421958923
[step: 486] loss: 0.3777587115764618
[step: 487] loss: 0.3776845335960388
[step: 488] loss: 0.37761035561561584
[step: 489] loss: 0.3775356113910675
[step: 490] loss: 0.3774603605270386
[step: 491] loss: 0.37738534808158875
[step: 492] loss: 0.37731093168258667
[step: 493] loss: 0.37723594903945923
[step: 494] loss: 0.3771606385707855
[step: 495] loss: 0.3770849406719208
[step: 496] loss: 0.3770095407962799
[step: 497] loss: 0.3769342601299286
[step: 498] loss: 0.376858651638031
[step: 499] loss: 0.3767828047275543
RMSE: 0.03070569969713688
